In [1]:
import re
import polars as pl
pl.Config.set_tbl_rows(None)

polars.config.Config

In [16]:
pattern = re.compile(
    r"\[info\] (?:\w+)\.([^\s:]+)(?::([\w\.\-]+))?\s+thrpt\s+\d+\s+(≈?\s*\S+)(?:\s±\s+([\d\.]+))?\s+(\S+)"
)

def unicode_superscript_to_float(s: str) -> float:
    """
    Converts a string like '10⁻⁶' or '≈ 10⁻⁵' into a float (e.g., 1e-6).
    """
    superscripts = {
        '⁰': '0', '¹': '1', '²': '2', '³': '3',
        '⁴': '4', '⁵': '5', '⁶': '6', '⁷': '7', '⁸': '8', '⁹': '9',
        '⁻': '-', '⁺': '+'
    }
    s = s.replace('≈', '').strip()
    if '10' in s:
        base, power = s.split('10')
        exp = ''.join(superscripts.get(ch, '') for ch in power)
        return float(f"1e{exp}")
    return float(s)

# Regex pattern for extracting rows
def parse_jmh(raw_jmh: str) -> pl.DataFrame:
    rows = []
    for m in pattern.finditer(raw_jmh):
        benchmark = m.group(1)
        metric = m.group(2) or "throughput"
        val_str = m.group(3).strip()
        try:
            value = float(val_str.replace("≈", ""))
        except ValueError:
            value = unicode_superscript_to_float(val_str)
        error = float(m.group(4)) if m.group(4) else None
        unit = m.group(5)
        rows.append((benchmark, metric, value, error, unit))


    # Create Polars DataFrame
    df = pl.DataFrame(rows, schema=["benchmark", "metric", "value", "error", "unit"])
    return df

shape: (21, 5)
┌──────────────────────────┬────────────────────┬────────────┬──────────┬────────┐
│ benchmark                ┆ metric             ┆ value      ┆ error    ┆ unit   │
│ ---                      ┆ ---                ┆ ---        ┆ ---      ┆ ---    │
│ str                      ┆ str                ┆ f64        ┆ f64      ┆ str    │
╞══════════════════════════╪════════════════════╪════════════╪══════════╪════════╡
│ matchAlternationFailLate ┆ throughput         ┆ 126.484    ┆ 4.582    ┆ ops/ms │
│ matchAlternationFailLate ┆ gc.alloc.rate      ┆ 0.001      ┆ 0.001    ┆ MB/sec │
│ matchAlternationFailLate ┆ gc.alloc.rate.norm ┆ 0.004      ┆ 0.001    ┆ B/op   │
│ matchAlternationFailLate ┆ gc.count           ┆ 0.0        ┆ null     ┆ counts │
│ matchAlternationHit      ┆ throughput         ┆ 205597.701 ┆ 5626.293 ┆ ops/ms │
│ …                        ┆ …                  ┆ …          ┆ …        ┆ …      │
│ matchBigClass            ┆ gc.time            ┆ 171.0      ┆ null     

/var/folders/vd/3qxwhj0s52bfzjbgt10xczn00000gn/T/ipykernel_89912/990238811.py:63: DataOrientationWarning: Row orientation inferred during DataFrame construction. Explicitly specify the orientation by passing `orient="row"` to silence this warning.
  df = pl.DataFrame(rows, schema=["benchmark", "metric", "value", "error", "unit"])


In [18]:
raw_jmh_prog = """
[info] BacktrackBenchmark.matchAlternationFailLate                     thrpt   10     126.484 ±    4.582  ops/ms
[info] BacktrackBenchmark.matchAlternationFailLate:gc.alloc.rate       thrpt   10       0.001 ±    0.001  MB/sec
[info] BacktrackBenchmark.matchAlternationFailLate:gc.alloc.rate.norm  thrpt   10       0.004 ±    0.001    B/op
[info] BacktrackBenchmark.matchAlternationFailLate:gc.count            thrpt   10         ≈ 0             counts
[info] BacktrackBenchmark.matchAlternationHit                          thrpt   10  205597.701 ± 5626.293  ops/ms
[info] BacktrackBenchmark.matchAlternationHit:gc.alloc.rate            thrpt   10       0.001 ±    0.001  MB/sec
[info] BacktrackBenchmark.matchAlternationHit:gc.alloc.rate.norm       thrpt   10      ≈ 10⁻⁶               B/op
[info] BacktrackBenchmark.matchAlternationHit:gc.count                 thrpt   10         ≈ 0             counts
[info] BacktrackBenchmark.matchBacktrackTrap                           thrpt   10   34317.522 ± 1503.136  ops/ms
[info] BacktrackBenchmark.matchBacktrackTrap:gc.alloc.rate             thrpt   10       0.001 ±    0.001  MB/sec
[info] BacktrackBenchmark.matchBacktrackTrap:gc.alloc.rate.norm        thrpt   10      ≈ 10⁻⁵               B/op
[info] BacktrackBenchmark.matchBacktrackTrap:gc.count                  thrpt   10         ≈ 0             counts
[info] BacktrackBenchmark.matchBigClass                                thrpt   10    4101.543 ±  281.476  ops/ms
[info] BacktrackBenchmark.matchBigClass:gc.alloc.rate                  thrpt   10   15400.969 ± 1050.835  MB/sec
[info] BacktrackBenchmark.matchBigClass:gc.alloc.rate.norm             thrpt   10    3944.000 ±    0.001    B/op
[info] BacktrackBenchmark.matchBigClass:gc.count                       thrpt   10     317.000             counts
[info] BacktrackBenchmark.matchBigClass:gc.time                        thrpt   10     171.000                 ms
[info] BacktrackBenchmark.matchFailLargeClass                          thrpt   10   88196.591 ± 1032.987  ops/ms
[info] BacktrackBenchmark.matchFailLargeClass:gc.alloc.rate            thrpt   10       0.001 ±    0.001  MB/sec
[info] BacktrackBenchmark.matchFailLargeClass:gc.alloc.rate.norm       thrpt   10      ≈ 10⁻⁵               B/op
[info] BacktrackBenchmark.matchFailLargeClass:gc.count                 thrpt   10         ≈ 0             counts
"""

prog_df = parse_jmh(raw_jmh_prog)
print(prog_df)

shape: (21, 5)
┌──────────────────────────┬────────────────────┬────────────┬──────────┬────────┐
│ benchmark                ┆ metric             ┆ value      ┆ error    ┆ unit   │
│ ---                      ┆ ---                ┆ ---        ┆ ---      ┆ ---    │
│ str                      ┆ str                ┆ f64        ┆ f64      ┆ str    │
╞══════════════════════════╪════════════════════╪════════════╪══════════╪════════╡
│ matchAlternationFailLate ┆ throughput         ┆ 126.484    ┆ 4.582    ┆ ops/ms │
│ matchAlternationFailLate ┆ gc.alloc.rate      ┆ 0.001      ┆ 0.001    ┆ MB/sec │
│ matchAlternationFailLate ┆ gc.alloc.rate.norm ┆ 0.004      ┆ 0.001    ┆ B/op   │
│ matchAlternationFailLate ┆ gc.count           ┆ 0.0        ┆ null     ┆ counts │
│ matchAlternationHit      ┆ throughput         ┆ 205597.701 ┆ 5626.293 ┆ ops/ms │
│ …                        ┆ …                  ┆ …          ┆ …        ┆ …      │
│ matchBigClass            ┆ gc.time            ┆ 171.0      ┆ null     

/var/folders/vd/3qxwhj0s52bfzjbgt10xczn00000gn/T/ipykernel_89912/990238811.py:63: DataOrientationWarning: Row orientation inferred during DataFrame construction. Explicitly specify the orientation by passing `orient="row"` to silence this warning.
  df = pl.DataFrame(rows, schema=["benchmark", "metric", "value", "error", "unit"])


In [19]:
raw_jmh_cps = """
[info] Benchmark                                                        Mode  Cnt       Score       Error   Units
[info] BacktrackBenchmark.matchAlternationFailLate                     thrpt   10     113.044 ±     2.584  ops/ms
[info] BacktrackBenchmark.matchAlternationFailLate:gc.alloc.rate       thrpt   10       1.725 ±     0.040  MB/sec
[info] BacktrackBenchmark.matchAlternationFailLate:gc.alloc.rate.norm  thrpt   10      16.005 ±     0.001    B/op
[info] BacktrackBenchmark.matchAlternationFailLate:gc.count            thrpt   10       2.000              counts
[info] BacktrackBenchmark.matchAlternationFailLate:gc.time             thrpt   10       3.000                  ms
[info] BacktrackBenchmark.matchAlternationHit                          thrpt   10  163264.809 ± 10929.579  ops/ms
[info] BacktrackBenchmark.matchAlternationHit:gc.alloc.rate            thrpt   10    2489.246 ±   169.564  MB/sec
[info] BacktrackBenchmark.matchAlternationHit:gc.alloc.rate.norm       thrpt   10      16.000 ±     0.001    B/op
[info] BacktrackBenchmark.matchAlternationHit:gc.count                 thrpt   10     144.000              counts
[info] BacktrackBenchmark.matchAlternationHit:gc.time                  thrpt   10      77.000                  ms
[info] BacktrackBenchmark.matchBacktrackTrap                           thrpt   10   33376.408 ±  2421.052  ops/ms
[info] BacktrackBenchmark.matchBacktrackTrap:gc.alloc.rate             thrpt   10     509.033 ±    36.999  MB/sec
[info] BacktrackBenchmark.matchBacktrackTrap:gc.alloc.rate.norm        thrpt   10      16.000 ±     0.001    B/op
[info] BacktrackBenchmark.matchBacktrackTrap:gc.count                  thrpt   10      61.000              counts
[info] BacktrackBenchmark.matchBacktrackTrap:gc.time                   thrpt   10      23.000                  ms
[info] BacktrackBenchmark.matchBigClass                                thrpt   10    4046.940 ±   384.816  ops/ms
[info] BacktrackBenchmark.matchBigClass:gc.alloc.rate                  thrpt   10   15271.827 ±  1459.557  MB/sec
[info] BacktrackBenchmark.matchBigClass:gc.alloc.rate.norm             thrpt   10    3960.000 ±     0.001    B/op
[info] BacktrackBenchmark.matchBigClass:gc.count                       thrpt   10     317.000              counts
[info] BacktrackBenchmark.matchBigClass:gc.time                        thrpt   10     182.000                  ms
[info] BacktrackBenchmark.matchFailLargeClass                          thrpt   10   63218.455 ± 10133.051  ops/ms
[info] BacktrackBenchmark.matchFailLargeClass:gc.alloc.rate            thrpt   10     964.343 ±   154.800  MB/sec
[info] BacktrackBenchmark.matchFailLargeClass:gc.alloc.rate.norm       thrpt   10      16.000 ±     0.001    B/op
[info] BacktrackBenchmark.matchFailLargeClass:gc.count                 thrpt   10     116.000              counts
[info] BacktrackBenchmark.matchFailLargeClass:gc.time                  thrpt   10      36.000                  ms
"""

cps_df = parse_jmh(raw_jmh_cps)
print(cps_df)

shape: (25, 5)
┌──────────────────────────┬────────────────────┬───────────┬───────────┬────────┐
│ benchmark                ┆ metric             ┆ value     ┆ error     ┆ unit   │
│ ---                      ┆ ---                ┆ ---       ┆ ---       ┆ ---    │
│ str                      ┆ str                ┆ f64       ┆ f64       ┆ str    │
╞══════════════════════════╪════════════════════╪═══════════╪═══════════╪════════╡
│ matchAlternationFailLate ┆ throughput         ┆ 113.044   ┆ 2.584     ┆ ops/ms │
│ matchAlternationFailLate ┆ gc.alloc.rate      ┆ 1.725     ┆ 0.04      ┆ MB/sec │
│ matchAlternationFailLate ┆ gc.alloc.rate.norm ┆ 16.005    ┆ 0.001     ┆ B/op   │
│ matchAlternationFailLate ┆ gc.count           ┆ 2.0       ┆ null      ┆ counts │
│ matchAlternationFailLate ┆ gc.time            ┆ 3.0       ┆ null      ┆ ms     │
│ …                        ┆ …                  ┆ …         ┆ …         ┆ …      │
│ matchFailLargeClass      ┆ throughput         ┆ 63218.455 ┆ 10133.051 

/var/folders/vd/3qxwhj0s52bfzjbgt10xczn00000gn/T/ipykernel_89912/990238811.py:63: DataOrientationWarning: Row orientation inferred during DataFrame construction. Explicitly specify the orientation by passing `orient="row"` to silence this warning.
  df = pl.DataFrame(rows, schema=["benchmark", "metric", "value", "error", "unit"])


In [20]:
raw_jmh_javaimpl = """
[info] Benchmark                                                        Mode  Cnt      Score     Error   Units
[info] BacktrackBenchmark.matchAlternationFailLate                     thrpt   10     26.803 ±   0.878  ops/ms
[info] BacktrackBenchmark.matchAlternationFailLate:gc.alloc.rate       thrpt   10    532.539 ±  17.473  MB/sec
[info] BacktrackBenchmark.matchAlternationFailLate:gc.alloc.rate.norm  thrpt   10  20840.021 ±   0.002    B/op
[info] BacktrackBenchmark.matchAlternationFailLate:gc.count            thrpt   10     63.000            counts
[info] BacktrackBenchmark.matchAlternationFailLate:gc.time             thrpt   10     24.000                ms
[info] BacktrackBenchmark.matchAlternationHit                          thrpt   10  20701.254 ± 538.638  ops/ms
[info] BacktrackBenchmark.matchAlternationHit:gc.alloc.rate            thrpt   10   4105.053 ± 106.697  MB/sec
[info] BacktrackBenchmark.matchAlternationHit:gc.alloc.rate.norm       thrpt   10    208.000 ±   0.001    B/op
[info] BacktrackBenchmark.matchAlternationHit:gc.count                 thrpt   10    201.000            counts
[info] BacktrackBenchmark.matchAlternationHit:gc.time                  thrpt   10     90.000                ms
[info] BacktrackBenchmark.matchBacktrackTrap                           thrpt   10   3663.638 ± 172.075  ops/ms
[info] BacktrackBenchmark.matchBacktrackTrap:gc.alloc.rate             thrpt   10   1536.835 ±  72.455  MB/sec
[info] BacktrackBenchmark.matchBacktrackTrap:gc.alloc.rate.norm        thrpt   10    440.000 ±   0.001    B/op
[info] BacktrackBenchmark.matchBacktrackTrap:gc.count                  thrpt   10    115.000            counts
[info] BacktrackBenchmark.matchBacktrackTrap:gc.time                   thrpt   10     43.000                ms
[info] BacktrackBenchmark.matchBigClass                                thrpt   10   9379.212 ± 190.450  ops/ms
[info] BacktrackBenchmark.matchBigClass:gc.alloc.rate                  thrpt   10   1144.686 ±  23.411  MB/sec
[info] BacktrackBenchmark.matchBigClass:gc.alloc.rate.norm             thrpt   10    128.000 ±   0.001    B/op
[info] BacktrackBenchmark.matchBigClass:gc.count                       thrpt   10    137.000            counts
[info] BacktrackBenchmark.matchBigClass:gc.time                        thrpt   10     36.000                ms
[info] BacktrackBenchmark.matchFailLargeClass                          thrpt   10  23999.821 ± 941.262  ops/ms
[info] BacktrackBenchmark.matchFailLargeClass:gc.alloc.rate            thrpt   10   2929.000 ± 115.151  MB/sec
[info] BacktrackBenchmark.matchFailLargeClass:gc.alloc.rate.norm       thrpt   10    128.000 ±   0.001    B/op
[info] BacktrackBenchmark.matchFailLargeClass:gc.count                 thrpt   10    154.000            counts
[info] BacktrackBenchmark.matchFailLargeClass:gc.time                  thrpt   10     74.000                ms
"""

javaimpl_df = parse_jmh(raw_jmh_javaimpl)

print(javaimpl_df)

shape: (25, 5)
┌──────────────────────────┬────────────────────┬───────────┬─────────┬────────┐
│ benchmark                ┆ metric             ┆ value     ┆ error   ┆ unit   │
│ ---                      ┆ ---                ┆ ---       ┆ ---     ┆ ---    │
│ str                      ┆ str                ┆ f64       ┆ f64     ┆ str    │
╞══════════════════════════╪════════════════════╪═══════════╪═════════╪════════╡
│ matchAlternationFailLate ┆ throughput         ┆ 26.803    ┆ 0.878   ┆ ops/ms │
│ matchAlternationFailLate ┆ gc.alloc.rate      ┆ 532.539   ┆ 17.473  ┆ MB/sec │
│ matchAlternationFailLate ┆ gc.alloc.rate.norm ┆ 20840.021 ┆ 0.002   ┆ B/op   │
│ matchAlternationFailLate ┆ gc.count           ┆ 63.0      ┆ null    ┆ counts │
│ matchAlternationFailLate ┆ gc.time            ┆ 24.0      ┆ null    ┆ ms     │
│ …                        ┆ …                  ┆ …         ┆ …       ┆ …      │
│ matchFailLargeClass      ┆ throughput         ┆ 23999.821 ┆ 941.262 ┆ ops/ms │
│ matchFailLa

/var/folders/vd/3qxwhj0s52bfzjbgt10xczn00000gn/T/ipykernel_89912/990238811.py:63: DataOrientationWarning: Row orientation inferred during DataFrame construction. Explicitly specify the orientation by passing `orient="row"` to silence this warning.
  df = pl.DataFrame(rows, schema=["benchmark", "metric", "value", "error", "unit"])
